# Loading packages and datasources

In [1]:
using Monkeypox
using Optim
using Optimization
using Plots
using Turing
using StatsPlots
using BSON:@save,@load
countryarray = ["United States", "Spain", "Germany", "England", "France", "Brazil", "Canada", "Netherlands"]
poparray = [329500000.0, 47350000.0, 83240000.0, 55980000.0, 67390000.0, 212600000.0, 38010000.0, 17440000.0]
url = "./data/timeseries-country-confirmed.csv"

"./data/timeseries-country-confirmed.csv"

# Bayesian inference

In [6]:
for (i, country) in enumerate(countryarray)
    data_on, acc, cases, datatspan, datadate = datasource!(url, country)
    bar(datadate,acc[datatspan],label="Accumulated cases",title = country)
    savefig("./output/accdata$i.png")
    bar(datadate, cases[datatspan], label="Daily cases", title=country)
    savefig("./output/dailydata$i.png")
    N = poparray[i] # population
    θ = [0.3, 0.3, 0.2, 0.1, 0.7, 0.01]# ρ,σ,h,α
    pknown = [0.0, 0.0, 1 / 30, 1.0] # B,μ,δ,ϕ
    lb = [0.0001, 0.0001, 0, 0.0001, 0.0001, 0.0]
    ub = [1.0, 1.0, 1.0, 1.0, 1.0, 0.1]
    alg = Optim.NelderMead()
    p_min = controlmonkeypoxopt!(N, θ, acc, cases, datatspan, pknown, lb, ub, alg)
    chainout = controlmonkeypoxinference!(N, p_min, acc, cases, datatspan, pknown, lb, ub)
    @save "chain$i.bason" chainout
    println(country, "data parameter:", chainout[2])
    prob_pred = controlmonkeypoxprob!(N, θ, acc, pknown)
    prediction = controlsimulate!(prob_pred, N, p_min, datatspan, pknown)
    scatter(datadate, acc[datatspan], label="Training data")
    display(plot!(datadate, prediction[10, :], label="Real accumulated cases"))
    savefig("./output/controlacc$i.png")
    #plot(datadate, prediction[2, :])
    #display(plot(chainout[1]))
    datatspan1 = 0:length(acc)-1
    prediction1 = controlsimulate!(prob_pred, N, p_min, datatspan1, pknown)
    mid = zeros(length(acc))
    mid[2:end] = prediction1[10, 1:end-1]
    pred_daily = prediction1[10, :] - mid
    scatter(datadate, cases[datatspan], label="Training data")
    display(plot!(datadate, pred_daily[datatspan], label="Daily cases", title=country))
    savefig("./output/controldaily$i.png")
    #plot(datadate, prediction[2, :])
    display(plot(chainout[1]))
    savefig("./output/controlchain$i.png")
end

: 

: 